In [1]:
import numpy as np
import os
#import h5py
import time
import copy
from random import randint
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(32, scale=(0.7, 1.0), ratio=(1.0,1.0)),
    transforms.ColorJitter(
            brightness=0.1*torch.randn(1),
            contrast=0.1*torch.randn(1),
            saturation=0.1*torch.randn(1),
            hue=0.1*torch.randn(1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)

Files already downloaded and verified


In [4]:
#Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)

class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3  ,196, 3,1,1)
        self.conv2 = nn.Conv2d(196,196, 3,2,1)
        self.conv3 = nn.Conv2d(196,196, 3,1,1)
        self.conv4 = nn.Conv2d(196,196, 3,2,1)
        self.conv5 = nn.Conv2d(196,196, 3,1,1)
        self.conv6 = nn.Conv2d(196,196, 3,1,1)
        self.conv7 = nn.Conv2d(196,196, 3,1,1)
        self.conv8 = nn.Conv2d(196,196, 3,2,1)
        
        self.ln1 = nn.LayerNorm((196,32,32))
        self.ln2 = nn.LayerNorm((196,16,16))
        self.ln3 = nn.LayerNorm((196,16,16))
        self.ln4 = nn.LayerNorm((196,8,8))
        self.ln5 = nn.LayerNorm((196,8,8))
        self.ln6 = nn.LayerNorm((196,8,8))
        self.ln7 = nn.LayerNorm((196,8,8))
        self.ln8 = nn.LayerNorm((196,4,4))
        
        
        
        self.pool = nn.MaxPool2d(4,4)           # 4*4
        self.lkrl = nn.LeakyReLU()
        self.fc1 = nn.Linear(196, 1)            # 1     critic output
        self.fc10 = nn.Linear(196, 10)          # 10    auxiliary 
        
    def forward(self, x):
        #layer1
        x = self.conv1(x)
        x = self.ln1(x)
        x = F.leaky_relu(x)
        
        x = self.conv2(x)
        x = self.ln2(x)
        x = F.leaky_relu(x)
       
        x = self.conv3(x)
        x = self.ln3(x)
        x = F.leaky_relu(x)
        
        x = self.conv4(x)
        x = self.ln4(x)
        x = F.leaky_relu(x)
        
        x = self.conv5(x)
        x = self.ln5(x)
        x = F.leaky_relu(x)
        
        x = self.conv6(x)
        x = self.ln6(x)
        x = F.leaky_relu(x)
        
        x = self.conv7(x)
        x = self.ln7(x)
        x = F.leaky_relu(x)
        
        x = self.conv8(x)
        x = self.ln8(x)
        x = F.leaky_relu(x)
        
        x = self.pool(x)
        x1 = x.view(-1, 196)
        critic = self.fc1(x1)
        x2 = x.view(-1, 196)
        classifier = self.fc10(x2)
        return critic,classifier

model =  discriminator().to(device)
#model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [5]:
#  TRAINING
epoch = 100
learning_rate = 0.0001
time1 = time.time()
#running_loss = 0.0
for epoch in range(epoch):  # loop over the dataset multiple times
    if(epoch==50):
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate/10.0
    if(epoch==75):
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate/100.0
    
    running_loss = 0.0
    for i, (X_train_batch, Y_train_batch) in enumerate(trainloader):

        if(Y_train_batch.shape[0] < batch_size):
            continue

        X_train_batch = Variable(X_train_batch).to(device)
        Y_train_batch = Variable(Y_train_batch).to(device)
        _, output = model(X_train_batch)

        loss = criterion(output, Y_train_batch)
        optimizer.zero_grad()
        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batch
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

        for group in optimizer.param_groups:
            for p in group['params']:
                state = optimizer.state[p]
                if('step' in state and state['step']>=1024):
                    state['step'] = 1000

        loss.backward()
        optimizer.step()
        
#if (epoch % 5 == 0):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            #images, labels = data
            images, labels = images.to(device), labels.to(device)

            _,outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Epoch',epoch+1 ,'Test accuracy: %f %%' % (
        100 * correct / total))


print('Finished Training')
torch.save(model,'cifar10.model')
time2 = time.time()
print(time2-time1)

[1,   100] loss: 1.949
[1,   200] loss: 1.610
[1,   300] loss: 1.505
Epoch 1 Test accuracy: 52.830000 %
[2,   100] loss: 1.328
[2,   200] loss: 1.274
[2,   300] loss: 1.219
Epoch 2 Test accuracy: 59.440000 %
[3,   100] loss: 1.068
[3,   200] loss: 1.012
[3,   300] loss: 1.014
Epoch 3 Test accuracy: 64.090000 %


KeyboardInterrupt: 

In [2]:
98 % 100


98